In [9]:
# import openpyxl module 
import openpyxl
from PIL import Image
import requests
from transformers import AutoProcessor, AutoModel
import torch
import matplotlib.pyplot as plt
import numpy as np

# SigLIP model
model = AutoModel.from_pretrained("google/siglip-base-patch16-224")
processor = AutoProcessor.from_pretrained("google/siglip-base-patch16-224")
  
# Give the location of the file 
path = "/Users/gabriellelittlefair/Documents/GitHub/MSc Project/Evaluation_Experiment/experiment.xlsx"
wb_obj = openpyxl.load_workbook(path) 
sheet_obj = wb_obj.active 

row = sheet_obj.max_row 
column = sheet_obj.max_column 

In [10]:
texts1 = ["a plan for a good layout of a bedroom", "a plan for a bad layout of a bedroom", "a plan for a layout of a bedroom"]
def SigLip_vals(filepath, texts):
    image = Image.open(filepath).convert('RGB')
    inputs = processor(text=texts, images=image, padding="max_length", return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = torch.sigmoid(logits_per_image) # these are the probabilities
    
    return probs.numpy()[0]

In [13]:
## SigLIP 
row = 25
texts1 = ["a plan for a good layout of a bedroom", "a plan for a bad layout of a bedroom", "a plan for a layout of a bedroom"]
texts2 = ["a plan for a good layout of a bedroom", "a plan for a bad layout of a bedroom", "a plan for a layout of a bedroom"]
texts3 = ["a top down view of a bedroom with a good layout", "a top down view of a bedroom with a bad layout", "a top down view of a bedroom with a layout"]
texts4 = ["a bedroom with a good layout", "a bedroom with a bad layout", "a bedroom with a layout"]
for i in range(1, row):
    
    v1 = SigLip_vals("/Users/gabriellelittlefair/Documents/GitHub/MSc Project/Evaluation_Experiment/bounding_boxes/bb_" + f"{i}.png", texts1)
    v2 = SigLip_vals("/Users/gabriellelittlefair/Documents/GitHub/MSc Project/Evaluation_Experiment/2Dtd/2Dtd_" + f"{i}.png", texts2)
    v3 = SigLip_vals("/Users/gabriellelittlefair/Documents/GitHub/MSc Project/Evaluation_Experiment/3Dtd/3Dtd_" + f"{i}.png", texts3)
    v4 = SigLip_vals("/Users/gabriellelittlefair/Documents/GitHub/MSc Project/Evaluation_Experiment/angled_view/av_" + f"{i}.png", texts4)
    sheet_obj.cell(row=i + 1, column=4).value = v1[0]
    sheet_obj.cell(row=i + 1, column=8).value = v2[0]
    sheet_obj.cell(row=i + 1, column=12).value = v3[0]
    sheet_obj.cell(row=i + 1, column=16).value = v4[0]
    sheet_obj.cell(row=i + 1, column=5).value = v1[1]
    sheet_obj.cell(row=i + 1, column=9).value = v2[1]
    sheet_obj.cell(row=i + 1, column=13).value = v3[1]
    sheet_obj.cell(row=i + 1, column=17).value = v4[1]
    sheet_obj.cell(row=i + 1, column=6).value = v1[2]
    sheet_obj.cell(row=i + 1, column=10).value = v2[2]
    sheet_obj.cell(row=i + 1, column=14).value = v3[2]
    sheet_obj.cell(row=i + 1, column=18).value = v4[2]

wb_obj.save("/Users/gabriellelittlefair/Documents/GitHub/MSc Project/Evaluation_Experiment/experiment.xlsx")